# Selecting and Filtering Data

```{admonition} Summary
:class: hint


```

For analyzing the data, if the data is huge, the processing with Jupyter will take more time, or sometimes the result will not be visualized. So the data could be limited to small parts by selecting the data of interest.

Load data first.

In [255]:
import geopandas as gp
import pandas as pd
from pathlib import Path

INPUT = Path.cwd().parents[0] / "00_data"
gdb_path = INPUT / "LBM_2018_IS_DD_gdb" / "LBM_2018_IS_DD.gdb"

gdf = gp.read_file(gdb_path, layer="LBM2018_IS_DD")

The headers (attributes) of the data are called.

In [256]:
list(gdf.columns)

['KS_IS',
 'CLC_st1',
 'CLC18',
 'CLC',
 'Biotpkt2018',
 'Shape_Length',
 'Shape_Area',
 'geometry']

It is important to have information about the format of each attribute. 

`````{admonition} Essentials of Attribute Formatting
:class: note
When working with attributes:
- Use quotes ('  ') for text or numbers stored as string (object) format
- For numeric values (like integers or floats), use them directly without quotes.

`````

In [257]:
gdf.dtypes

KS_IS             object
CLC_st1           object
CLC18             object
CLC              float64
Biotpkt2018      float64
Shape_Length     float64
Shape_Area       float64
geometry        geometry
dtype: object

```{figure} ../resources/22.png
:name: figure-example

Attributes in Dataset Based on the Metadata
````


## Unique Values Selection


The `unique` function returns all distinct values of an attribute.


**ex:** In this dataset, the `CLC_st1` attribute might have some identical values for land use (such as '313' for mixed forest or '512' for water bodies) but multiple features in the dataset recorded as mixed forest or water bodies.

```{figure} ../resources/19.png
:name: figure-example

Classification Code Interpretation
````


By defining the attribute of interest in this example "KS_IS", the unique values in this attribute are visualized.

In [258]:
unique_groups = gdf['KS_IS'].unique()
unique_groups

array(['SV', 'HW', 'BV', 'FV', 'WW', 'WP', 'FG', 'GT', 'FL', 'BN', 'BNL',
       'BL', 'HE', None], dtype=object)

```{figure} ../resources/21.png
:name: figure-example
Interpretation of the values in the "KS_IS" attribute
````


Also the number of the unique values can be accessed by `nunique` function.

In [259]:
unique_numbers = gdf['KS_IS'].nunique()
unique_numbers

13

`````{admonition} None values in nunique function!
:class: danger

By default, the `nunique` function does not count **null** or **missing** values.
`````

For including the `None` values as unique values, `dropna` parameter should modify to False.

In [260]:
unique_numbers = gdf['KS_IS'].nunique(dropna = False)
unique_numbers

14

The frequency of each unique value can be counted using the `groupby` method.


The data then is grouped by unique values, and the number of entries in each group is calculated using the `size` function.

In the following example, just 1 feature recorded for the category of BNL, while the category of WP has 1768 records in this dataset.

`````{admonition} Order of Groups with size function
:class: tip, dropdown
The result of `size` function is sorted based on unique values:
- For **string** unique values, sorting is in **alphabetical** order.
- For **number** unique values, sorting is in **ascending numerical** order.


**ex:** In the following example, the unique values are in the string format so they ordered alphabetically as BL, BN, ...
`````


In [261]:
groupcounts = gdf.groupby('KS_IS').size()
pd.DataFrame(groupcounts).T

KS_IS,BL,BN,BNL,BV,FG,FL,FV,GT,HE,HW,SV,WP,WW
0,640,4,1,13,1737,36,2,391,22,1194,271,1768,575


The frequency of unique values can also be counted using the `value_counts` method.


`````{admonition} Order of Groups with value_counts function
:class: tip, dropdown
The output of the `value_counts` function is sorted in descending order based on the frequency counts.


**ex:** In the following example the counts ordered descending.
`````


In [262]:
groupcounts = gdf["KS_IS"].value_counts()
pd.DataFrame(groupcounts).T

KS_IS,WP,FG,HW,BL,WW,GT,SV,FL,HE,BV,BN,FV,BNL
count,1768,1737,1194,640,575,391,271,36,22,13,4,2,1


In a long list of unique values, if the number of features for a specific group is needed, it can be accessed like a value for a key in a dictionary.

In the following example, the output shows that in this dataset 271 features recorded as SV (representing roads).

In [263]:
sv_count = groupcounts['SV']
sv_count

271

`````{admonition} Format Importance Use Case
:class: warning,
The importance of attribute formatting, mentioned at the beginning of this chapter, will be highlighted here. In the previous example, the values were text (string), so for calling the values, the quotes were used. On the other hand, in the following examples, the values for both attributes of "CLC_st1" and "Biotpkt2018" are numbers, while the format of the first is an object (string) and the second is a float.
`````

- **String format**
- Below, we also sort values (`.sort_values()`)
- and only return the first `[:10]` entries

In [264]:
group_1 = gdf.groupby('CLC_st1').size()
group_1.sort_values(ascending=False)[:10]

CLC_st1
112    7187
122    5454
231    3293
121    2732
511    2563
142    2413
311    1787
141    1627
313    1594
312    1546
dtype: int64

Access a specific index value:

In [265]:
group_1['141']

1627

- **Float format**

In [266]:
group_2 = gdf.groupby('Biotpkt2018').size()
group_2

Biotpkt2018
0.000000      16
0.500000      55
1.000000      52
1.500000     102
2.000000     131
            ... 
16.928546     36
17.079406      3
17.381302     10
18.055116    667
20.164493      8
Length: 69, dtype: int64

In [267]:
group_2[1.000000]

52

```{admonition} Importance of the precision of float format!
:class: note, dropdown
Sometimes, due to the precision of the float format, the printed output of the `groupby` or `value_counts` function may not exactly match the value in the DataFrame. In this case, if a value from their output is called directly, an error will occur.

The following figure is an example of this note, when the value "18.055116" called. This value is an output of the `groupby` function in the previous step, but it is a rounded value, and this value in the dataset includes additional decimal places like 18.05511611 or 18.05511689 or ...

```{figure} ../resources/14_.png
:width: 700px
:name: figure-example 
Error of calling rounded values instead of precise values in the dataset
```


## Selection of String Values

After becoming familiar with attributes and unique values of the features, the required features can be selected:

If only features with a specific value are needed, the value can be called directly:


In [268]:
filtered_data = gdf[gdf['KS_IS'] == 'FG']
filtered_data.head(5)

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry
3823,FG,511,None,NaN,12.594189,491.915386,1070.836783,"MULTIPOLYGON (((415374.978 5648049.894, 415371..."
3824,FG,511,None,NaN,12.594189,268.168647,581.000813,"MULTIPOLYGON (((415477.458 5648133.623, 415476..."
3825,FG,511,None,NaN,12.594189,16.334540,15.163151,"MULTIPOLYGON (((415489.003 5648142.273, 415488..."
3826,FG,511,None,NaN,12.594189,1436.401563,3210.514272,"MULTIPOLYGON (((415773.656 5648659.571, 415773..."
3827,FG,511,None,NaN,12.594189,399.339616,877.628801,"MULTIPOLYGON (((415930.38 5648763.655, 415926...."


In [269]:
filtered_data = gdf[gdf['CLC_st1'] == '133']
filtered_data.head(5)

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228..."
24343,None,133,133,133.0,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664...."
24344,None,133,133,133.0,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562..."
24347,None,133,133,133.0,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588..."


## Selection of Float Values


In [270]:
filtered_data = gdf[gdf['Biotpkt2018'] == 1.000000]
filtered_data.head(5)

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry
9492,None,111,111,111.0,1.0,411.855290,2480.458787,"MULTIPOLYGON (((406985.474 5655597.694, 406984..."
9504,None,111,111,111.0,1.0,89.433492,466.944726,"MULTIPOLYGON (((416782.224 5652196.728, 416782..."
9514,None,111,111,111.0,1.0,172.927119,1621.632535,"MULTIPOLYGON (((419300.05 5657313.72, 419299.9..."
9515,None,111,111,111.0,1.0,320.578260,3691.824728,"MULTIPOLYGON (((408302.234 5655785.262, 408296..."
9519,None,111,111,111.0,1.0,121.899927,920.751286,"MULTIPOLYGON (((414687.929 5655817.21, 414656...."


`````{admonition} Importance of the precision of float format!
:class: note, dropdown
As mentioned in the previous note, for float values, due to precision, it may sometimes show an empty DataFrame (meaning no feature with this value exists), while in `groupby` or `value_counts` functions, it may indicate that some features with this value exist.

```{figure} ../resources/15_.png
:width: 600px
:name: figure-example
Empty output for calling rounded values instead of precise values in the dataset
`````

In [271]:
# Codes of the upper note
filtered_data = gdf[gdf['Biotpkt2018'] == 18.055116]
filtered_data.head(5)

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry


If features with more than one specific value are needed, the values can be called as a list using `isin` method.

In [272]:
filtered_data = gdf[gdf['CLC_st1'].isin(['133', '321', '411'])]
filtered_data.head(5)

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228..."
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579..."
23860,None,321,321,321.0,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618...."
23861,None,321,321,321.0,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516..."
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151..."


## Adding New column

To add a new column to a dataset, the column name is defined for the dataset, and values are assigned to each row.

In the following example, the output of the previous step is used as the dataset, and a new column named "test" is added. 

`````{admonition} [:, 'column_name']
:class: attention, dropdown

Using `:` sign before `,` means selecting all the rows, and the information after `,` specifies the columns of interest.
`````

Since the dataset contains 15 rows, 15 values are assigned using the `loc` method. (The values could be from different formats: string, null, negative, float, integer, ...)

In [273]:
filtered_data.loc[:, 'test'] = ['Hello', None, -7, 4.5, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  
filtered_data.head()

/opt/conda/envs/worker_env/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",Hello
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",None
23860,None,321,321,321.0,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",-7
23861,None,321,321,321.0,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",4.5
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",5


If there are observed values only for some rows while the rest are missing:

First, the **None** value is assigned to all rows using `loc` method.

In [274]:
filtered_data.loc[:,'test'] = None

Then, the rows are updated with the observed values using `loc` method.

For updating the rows, the index of the rows is required.

The index of the rows can be identified either by printing the dataset or by using the `index` method.

In [275]:
print(filtered_data.index)

Index([11831, 23859, 23860, 23861, 23862, 23863, 23864, 24343, 24344, 24347,
       24444, 24445, 32673, 32674, 33881],
      dtype='int64')


In the following example, the index 11831, 23859, and 23862 are updated with their new values.

In [276]:
filtered_data.loc[[11831, 23859, 23862], 'test'] = [100, 200, '300']

In [277]:
filtered_data.head()

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200
23860,None,321,321,321.0,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None
23861,None,321,321,321.0,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300


In [278]:
https://stag.training.fdz.ioer.info/notebooks/309_merging_data.html#merging-based-on-common-attributes-using-merge

SyntaxError: invalid decimal literal (2176582010.py, line 1)

## Adding New Row

To create a new feature (row) in the dataset, the `loc` method is used.

In [279]:
Here the index of the row is defined and for all the columns (CLC_st1, Biotpkt2018, ...) the values should be assigned.

SyntaxError: invalid syntax (1313456832.py, line 1)

In [280]:
filtered_data.loc['sample'] = ['Hello', 800, -7, 4.5, 5, 6, 7, None, 9]
filtered_data.head(16)

/tmp/ipykernel_28639/1062012079.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  filtered_data.loc['sample'] = ['Hello', 800, -7, 4.5, 5, 6, 7, None, 9]
/tmp/ipykernel_28639/1062012079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc['sample'] = ['Hello', 800, -7, 4.5, 5, 6, 7, None, 9]


,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200
23860,None,321,321,321.0,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None
23861,None,321,321,321.0,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300
23863,None,321,321,321.0,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None
23864,None,321,321,321.0,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None
24343,None,133,133,133.0,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None
24344,None,133,133,133.0,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None
24347,None,133,133,133.0,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None


In [281]:
https://stag.training.fdz.ioer.info/notebooks/309_merging_data.html#merging-similar-datasets-using-concat

SyntaxError: invalid decimal literal (3468439655.py, line 1)

## Copy a Column

Copying a column in a dataset is the same as adding a new column, except that instead of defining values, the values from the column of interest are imported.

In the example below, the values from the `test` column are copied into the `test_2` column. 

In [282]:
filtered_data.loc[:,'test_2'] = filtered_data.loc[:,'test']
filtered_data.head(17)

/opt/conda/envs/worker_env/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23860,None,321,321,321.0,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None,None
23861,None,321,321,321.0,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None,None
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
23863,None,321,321,321.0,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None,None
23864,None,321,321,321.0,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None,None
24343,None,133,133,133.0,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None,None
24344,None,133,133,133.0,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None,None
24347,None,133,133,133.0,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None,None


## Copy a Row 

Copying a row in a dataset is the same as adding a new row, except that instead of defining values, the values from the row of interest are imported.

In the example below, the values from the `sample` row are copied into the `sample_2` row. 

In [283]:
filtered_data.loc['sample_2'] = filtered_data.loc['sample']
filtered_data.head(18)

/tmp/ipykernel_28639/3211081606.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  filtered_data.loc['sample_2'] = filtered_data.loc['sample']
/tmp/ipykernel_28639/3211081606.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc['sample_2'] = filtered_data.loc['sample']


,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23860,None,321,321,321.0,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None,None
23861,None,321,321,321.0,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None,None
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
23863,None,321,321,321.0,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None,None
23864,None,321,321,321.0,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None,None
24343,None,133,133,133.0,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None,None
24344,None,133,133,133.0,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None,None
24347,None,133,133,133.0,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None,None


## Finding Null Values

To find the columns that include null values:
- `isnull`: Provide the same dataset but with the True (if it is a null value) or False value.
- `any`: Check if the columns include at least one True value.

In [284]:
filtered_data.isnull().any()

KS_IS            True
CLC_st1         False
CLC18           False
CLC             False
Biotpkt2018     False
Shape_Length    False
Shape_Area      False
geometry         True
test             True
test_2           True
dtype: bool

Then for the columns including null values, the related index is extracted.

In [285]:
filtered_data[filtered_data['test'].isnull()].index

Index([23860, 23861, 23863, 23864, 24343, 24344, 24347, 24444, 24445, 32673,
       32674, 33881],
      dtype='object')

Also it is possible to extract all the rows that include at least one null value.

- `isnull`: Provide the same dataset but with the True (if it is a null value) or False value.
- `any(axis=1)`: **axis = 1** checks for the True values in the **rows** of dataset. 

In [286]:
filtered_data[filtered_data.isnull().any(axis=1)]

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23860,None,321,321,321.0,20.164493,1099.972947,10894.472486,"MULTIPOLYGON (((418611.74 5653542.485, 418618....",None,None
23861,None,321,321,321.0,20.164493,1023.775893,22918.685826,"MULTIPOLYGON (((418506.559 5653953.337, 418516...",None,None
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
23863,None,321,321,321.0,20.164493,1251.411517,28498.211647,"MULTIPOLYGON (((401726.203 5658676.434, 401724...",None,None
23864,None,321,321,321.0,20.164493,1055.657824,15517.253917,"MULTIPOLYGON (((403428.145 5659215.144, 403416...",None,None
24343,None,133,133,133.0,6.035751,287.703701,4278.338534,"MULTIPOLYGON (((411656.913 5656302.92, 411664....",None,None
24344,None,133,133,133.0,6.035751,183.544225,2066.507754,"MULTIPOLYGON (((411567.287 5656337.438, 411562...",None,None
24347,None,133,133,133.0,6.035751,213.015432,2684.440542,"MULTIPOLYGON (((411587.489 5655959.618, 411588...",None,None


## Removing Null Values

To remove the rows that include `null` values in a specific column, the `dropna` method is used.

In [297]:
filtered_data.dropna()

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2


To remove the rows that include `null` values in a specific column, the `dropna` method is used.

The name of the column(s) are defined in the `subset` parameter.

In [298]:
filtered_data.dropna(subset=['test'])

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry,test,test_2
11831,None,133,133,133.0,6.035751,17.828953,0.432276,"MULTIPOLYGON (((411237.019 5656164.663, 411228...",100,100
23859,None,321,321,321.0,20.164493,618.702965,18317.038350,"MULTIPOLYGON (((415627.211 5652243.292, 415579...",200,200
23862,None,321,321,321.0,20.164493,1126.678849,17980.357862,"MULTIPOLYGON (((418145.759 5654361.049, 418151...",300,300
sample,Hello,800,-7,4.5,5.000000,6.000000,7.000000,None,9,9
sample_2,Hello,800,-7,4.5,5.000000,6.000000,7.000000,None,9,9


To remove columns with null values the `dropna` method is used with the `axis=1` parameter. 

In [299]:
filtered_data.dropna(axis=1)

,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area
11831,133,133,133.0,6.035751,17.828953,0.432276
23859,321,321,321.0,20.164493,618.702965,18317.038350
23860,321,321,321.0,20.164493,1099.972947,10894.472486
23861,321,321,321.0,20.164493,1023.775893,22918.685826
23862,321,321,321.0,20.164493,1126.678849,17980.357862
23863,321,321,321.0,20.164493,1251.411517,28498.211647
23864,321,321,321.0,20.164493,1055.657824,15517.253917
24343,133,133,133.0,6.035751,287.703701,4278.338534
24344,133,133,133.0,6.035751,183.544225,2066.507754
24347,133,133,133.0,6.035751,213.015432,2684.440542


In [254]:
removing the whole column

SyntaxError: invalid syntax (315404214.py, line 1)

In [31]:
filtered_data = filtered_data.drop('test', axis=1)
filtered_data.head()

KeyError: "['new_column'] not found in axis"

In [ ]:
removing the whole row

In [ ]:
####Null values

In [ ]:
#####

In [ ]:
######

It is also possible to explore the range of the data for numeric values, such as area.


To find the minimum and maximum values of the features, the following code is used:
- `min`: minimum value in the specified attribute
- `max`: maximum value in the specified attribute
- `:0.8f`: `:f` treated the value as a float, and `0.8` rounded the number of decimals to 8 numbers



`````{admonition} 
:class: tip
For more controlling over the output visit[here](https://python.aims.ac.za/pages/format_str.html#id4)
`````



In [29]:
min_value = gdf['Shape_Area'].min()
max_value = gdf['Shape_Area'].max()
print(f"minimum: {min_value:0.8f}")
print(f"maximum: {max_value:0.2f}")

minimum: 0.00000562
maximum: 3249895.12


Now that information about the attributes of features and the values assigned to them has been obtained, it is possible to decide which part of the data is of interest.


For example, it can be decided to work only with the features having areas less than 1000.


In [36]:
filter_db = gdf[gdf['Shape_Area'] < 1000]

In [39]:
filter_db.head()

,KS_IS,CLC_st1,CLC18,CLC,Biotpkt2018,Shape_Length,Shape_Area,geometry
1,SV,122,None,NaN,5.271487,31.935928,50.075513,"MULTIPOLYGON (((417850.525 5650376.33, 417846...."
3,SV,122,None,NaN,5.271487,24.509066,36.443441,"MULTIPOLYGON (((423453.146 5650332.06, 423453...."
4,SV,122,None,NaN,5.271487,29.937138,40.494155,"MULTIPOLYGON (((417331.434 5650889.039, 417330..."
6,SV,122,None,NaN,5.271487,390.293053,764.349833,"MULTIPOLYGON (((407613.154 5651943.2, 407618.7..."
7,SV,122,None,NaN,5.271487,34.931619,45.486260,"MULTIPOLYGON (((416066.508 5651463.834, 416066..."


In the next chapter, it is explained how to plot this data.